# PySpark DataFrame 高级操作

## 数据清洗与转换实战教程

本教程涵盖：
- 🧹 数据清洗（缺失值、重复值、异常值）
- 🔄 数据转换（类型转换、列操作）
- 🎯 高级选择与过滤
- 🔗 复杂数据类型处理
- 🛠️ 实用技巧与最佳实践

In [ ]:
# 环境设置
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

# 创建 Spark Session
spark = SparkSession.builder \
    .appName("DataFrame 高级操作") \
    .master("local[*]") \
    .config("spark.driver.memory", "2g") \
    .getOrCreate()

print(f"✅ Spark {spark.version} 已启动")
print(f"✅ Spark UI: http://localhost:4040")

## 第一部分：创建模拟数据

创建一个包含各种数据质量问题的数据集，模拟真实场景。

In [ ]:
# 创建包含数据质量问题的用户数据
messy_data = [
    (1, "Alice", "alice@email.com", 28, "Female", 75000.0, "2023-01-15", "New York"),
    (2, "Bob", "bob@email.com", 35, "Male", 85000.0, "2023-02-20", "Los Angeles"),
    (3, "Charlie", None, None, "Male", 65000.0, "2023-03-10", "Chicago"),  # 缺失邮箱和年龄
    (4, "David", "david@email.com", 42, "Male", None, "2023-04-05", "Houston"),  # 缺失薪资
    (5, None, "eve@email.com", 29, "Female", 70000.0, None, "Phoenix"),  # 缺失姓名和日期
    (6, "Frank", "frank@invalid", -5, "Male", 72000.0, "2023-06-15", "Philadelphia"),  # 异常年龄
    (7, "Grace", "grace@email.com", 31, "Female", 78000.0, "2023-07-20", "San Antonio"),
    (2, "Bob", "bob@email.com", 35, "Male", 85000.0, "2023-02-20", "Los Angeles"),  # 重复行
    (8, "Henry", "henry@email.com", 150, "Male", 90000.0, "2023-08-25", "San Diego"),  # 异常年龄
    (9, "Ivy", "ivy@email.com", 26, "Female", -10000.0, "2023-09-30", "Dallas"),  # 异常薪资
    (10, "Jack", "JACK@EMAIL.COM", 33, "male", 82000.0, "2023-10-15", "san jose"),  # 大小写不一致
]

columns = ["id", "name", "email", "age", "gender", "salary", "join_date", "city"]

df = spark.createDataFrame(messy_data, columns)

print("原始数据（包含质量问题）：")
df.show(truncate=False)

print("\n数据概览：")
df.printSchema()
print(f"总行数: {df.count()}")

## 第二部分：数据质量检查

在清洗之前，先了解数据的问题。

In [ ]:
# 1. 检查每列的缺失值数量
print("=" * 60)
print("缺失值统计")
print("=" * 60)

from pyspark.sql.functions import col, sum as _sum, count, when, isnan

# 统计每列的 NULL 值
null_counts = df.select([
    _sum(when(col(c).isNull(), 1).otherwise(0)).alias(c)
    for c in df.columns
])

print("每列的 NULL 值数量：")
null_counts.show()

# 2. 检查重复行
print("\n" + "=" * 60)
print("重复数据检查")
print("=" * 60)

total_rows = df.count()
distinct_rows = df.distinct().count()
duplicate_rows = total_rows - distinct_rows

print(f"总行数: {total_rows}")
print(f"唯一行数: {distinct_rows}")
print(f"重复行数: {duplicate_rows}")

# 3. 基本统计信息
print("\n" + "=" * 60)
print("数值列统计")
print("=" * 60)

df.select("age", "salary").describe().show()

## 第三部分：数据清洗

### 3.1 处理重复数据

In [ ]:
# 方法 1：删除完全重复的行
df_no_duplicates = df.dropDuplicates()

print("删除重复行后：")
print(f"原始行数: {df.count()} → 清洗后: {df_no_duplicates.count()}")

# 方法 2：基于特定列删除重复（保留第一条）
df_no_dup_id = df.dropDuplicates(["id"])

print(f"\n基于 ID 删除重复后: {df_no_dup_id.count()} 行")

# 使用清洗后的数据继续
df_clean = df_no_duplicates

### 3.2 处理缺失值

In [ ]:
print("=" * 60)
print("处理缺失值")
print("=" * 60)

# 方法 1：删除包含任何 NULL 的行
df_dropna_any = df_clean.dropna(how='any')
print(f"删除任何包含 NULL 的行: {df_dropna_any.count()} 行")

# 方法 2：删除所有列都是 NULL 的行
df_dropna_all = df_clean.dropna(how='all')
print(f"删除全部为 NULL 的行: {df_dropna_all.count()} 行")

# 方法 3：删除特定列为 NULL 的行
df_dropna_subset = df_clean.dropna(subset=["name", "email"])
print(f"删除 name 或 email 为 NULL 的行: {df_dropna_subset.count()} 行")

# 方法 4：填充缺失值
print("\n填充缺失值示例：")

# 用固定值填充
df_filled = df_clean.fillna({
    "name": "Unknown",
    "email": "no-email@example.com",
    "age": 0,
    "salary": 0.0,
    "join_date": "1900-01-01"
})

print("用固定值填充后：")
df_filled.show(truncate=False)

# 方法 5：用平均值/中位数填充数值列
from pyspark.sql.functions import mean, median, approx_count_distinct

# 计算平均年龄和薪资
avg_age = df_clean.select(mean("age")).first()[0]
avg_salary = df_clean.select(mean("salary")).first()[0]

df_filled_avg = df_clean.fillna({
    "age": avg_age,
    "salary": avg_salary
})

print(f"\n用平均值填充: 年龄={avg_age:.1f}, 薪资={avg_salary:.2f}")
df_filled_avg.show(truncate=False)

### 3.3 处理异常值

In [ ]:
print("=" * 60)
print("处理异常值")
print("=" * 60)

# 1. 识别异常值（年龄应该在 18-100 之间）
df_age_check = df_filled_avg.withColumn(
    "age_valid",
    when((col("age") >= 18) & (col("age") <= 100), True).otherwise(False)
)

print("年龄异常值检查：")
df_age_check.select("name", "age", "age_valid").show()

# 2. 过滤异常值
df_no_outliers = df_filled_avg.filter(
    (col("age") >= 18) & (col("age") <= 100) &
    (col("salary") >= 0)
)

print(f"\n过滤异常值后: {df_no_outliers.count()} 行")
df_no_outliers.show(truncate=False)

# 3. 使用统计方法识别异常值（IQR 方法）
from pyspark.sql.functions import percentile_approx

# 计算四分位数
quantiles = df_filled_avg.approxQuantile("salary", [0.25, 0.75], 0.05)
Q1 = quantiles[0]
Q3 = quantiles[1]
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

print(f"\n薪资异常值检测（IQR 方法）：")
print(f"Q1 (25%): {Q1:.2f}")
print(f"Q3 (75%): {Q3:.2f}")
print(f"IQR: {IQR:.2f}")
print(f"正常范围: [{lower_bound:.2f}, {upper_bound:.2f}]")

df_iqr_filtered = df_filled_avg.filter(
    (col("salary") >= lower_bound) & (col("salary") <= upper_bound)
)

print(f"\n使用 IQR 过滤后: {df_iqr_filtered.count()} 行")

## 第四部分：数据转换

### 4.1 列操作（添加、删除、重命名）

In [ ]:
print("=" * 60)
print("列操作")
print("=" * 60)

# 1. 添加新列
df_with_cols = df_no_outliers \
    .withColumn("salary_category", 
                when(col("salary") < 70000, "Junior")
                .when(col("salary") < 80000, "Mid")
                .otherwise("Senior")) \
    .withColumn("age_group",
                when(col("age") < 30, "20s")
                .when(col("age") < 40, "30s")
                .otherwise("40+")) \
    .withColumn("full_name_upper", upper(col("name")))

print("添加新列后：")
df_with_cols.select("name", "age", "salary", "age_group", "salary_category").show()

# 2. 删除列
df_dropped = df_with_cols.drop("full_name_upper")
print(f"\n删除列后: {len(df_dropped.columns)} 列")

# 3. 重命名列
df_renamed = df_with_cols \
    .withColumnRenamed("name", "employee_name") \
    .withColumnRenamed("salary", "annual_salary")

print("\n重命名列后：")
print(df_renamed.columns)

### 4.2 字符串处理

In [ ]:
print("=" * 60)
print("字符串处理")
print("=" * 60)

# 1. 大小写转换
df_string = df_no_outliers \
    .withColumn("name_upper", upper(col("name"))) \
    .withColumn("name_lower", lower(col("name"))) \
    .withColumn("email_lower", lower(col("email"))) \
    .withColumn("city_proper", initcap(col("city")))  # 首字母大写

print("字符串转换：")
df_string.select("name", "name_upper", "name_lower", "city", "city_proper").show(truncate=False)

# 2. 去除空格
df_trimmed = df_no_outliers \
    .withColumn("name_trimmed", trim(col("name"))) \
    .withColumn("email_trimmed", trim(col("email")))

# 3. 字符串分割
df_split = df_no_outliers \
    .withColumn("email_parts", split(col("email"), "@")) \
    .withColumn("email_user", split(col("email"), "@")[0]) \
    .withColumn("email_domain", split(col("email"), "@")[1])

print("\n字符串分割（邮箱）：")
df_split.select("email", "email_user", "email_domain").show(truncate=False)

# 4. 字符串拼接
df_concat = df_no_outliers \
    .withColumn("full_info", 
                concat_ws(" - ", col("name"), col("city"), col("gender")))

print("\n字符串拼接：")
df_concat.select("full_info").show(truncate=False)

# 5. 正则表达式替换
df_regex = df_no_outliers \
    .withColumn("email_masked",
                regexp_replace(col("email"), "@.*", "@***"))

print("\n邮箱脱敏：")
df_regex.select("email", "email_masked").show(truncate=False)

### 4.3 类型转换

In [ ]:
print("=" * 60)
print("类型转换")
print("=" * 60)

# 查看当前数据类型
print("原始数据类型：")
df_no_outliers.printSchema()

# 1. 转换日期类型
df_typed = df_no_outliers \
    .withColumn("join_date", to_date(col("join_date"), "yyyy-MM-dd"))

# 2. 转换数值类型
df_typed = df_typed \
    .withColumn("age", col("age").cast(IntegerType())) \
    .withColumn("salary", col("salary").cast(DoubleType()))

print("\n转换后的数据类型：")
df_typed.printSchema()

# 3. 从日期中提取信息
df_date = df_typed \
    .withColumn("year", year(col("join_date"))) \
    .withColumn("month", month(col("join_date"))) \
    .withColumn("day", dayofmonth(col("join_date"))) \
    .withColumn("day_of_week", dayofweek(col("join_date"))) \
    .withColumn("quarter", quarter(col("join_date")))

print("\n日期信息提取：")
df_date.select("name", "join_date", "year", "month", "day", "quarter").show()

### 4.4 条件转换和 CASE WHEN

In [ ]:
print("=" * 60)
print("条件转换 (CASE WHEN)")
print("=" * 60)

# 1. 简单条件
df_conditions = df_typed \
    .withColumn("is_senior",
                when(col("age") >= 35, "Yes").otherwise("No")) \
    .withColumn("salary_level",
                when(col("salary") >= 85000, "High")
                .when(col("salary") >= 75000, "Medium-High")
                .when(col("salary") >= 70000, "Medium")
                .otherwise("Low"))

print("条件分类：")
df_conditions.select("name", "age", "is_senior", "salary", "salary_level").show()

# 2. 复杂条件组合
df_complex = df_typed \
    .withColumn("talent_category",
                when((col("age") < 30) & (col("salary") >= 75000), "Rising Star")
                .when((col("age") >= 35) & (col("salary") >= 80000), "Senior Expert")
                .when(col("salary") >= 85000, "High Performer")
                .otherwise("Regular"))

print("\n人才分类：")
df_complex.select("name", "age", "salary", "talent_category").show()

# 3. 使用 SQL 表达式
df_typed.createOrReplaceTempView("employees")

result = spark.sql("""
    SELECT 
        name,
        age,
        salary,
        CASE
            WHEN salary >= 85000 THEN '💰 高薪'
            WHEN salary >= 75000 THEN '💵 中高薪'
            ELSE '💴 普通'
        END as salary_emoji
    FROM employees
""")

print("\nSQL 条件表达式：")
result.show(truncate=False)

## 第五部分：高级选择与过滤

### 5.1 复杂过滤条件

In [ ]:
print("=" * 60)
print("高级过滤")
print("=" * 60)

# 1. 多条件过滤
filtered_1 = df_typed.filter(
    (col("age") >= 30) & 
    (col("salary") >= 75000) & 
    (col("gender") == "Female")
)

print("30岁以上、薪资>=75000 的女性：")
filtered_1.show()

# 2. IN 操作
filtered_2 = df_typed.filter(col("city").isin(["New York", "Los Angeles", "Chicago"]))

print("\n特定城市的员工：")
filtered_2.show()

# 3. LIKE 模糊匹配
filtered_3 = df_typed.filter(col("email").like("%@email.com"))

print("\n邮箱包含 @email.com 的员工：")
filtered_3.show()

# 4. 正则表达式匹配
filtered_4 = df_typed.filter(col("name").rlike("^[A-C].*"))  # 名字以 A、B、C 开头

print("\n名字以 A/B/C 开头的员工：")
filtered_4.show()

# 5. BETWEEN
filtered_5 = df_typed.filter(col("salary").between(70000, 80000))

print("\n薪资在 70000-80000 之间：")
filtered_5.show()

### 5.2 动态列选择

In [ ]:
print("=" * 60)
print("动态列选择")
print("=" * 60)

# 1. 选择特定列
selected_1 = df_typed.select("name", "age", "salary")
print("基本选择：")
selected_1.show(3)

# 2. 使用表达式选择
selected_2 = df_typed.select(
    col("name"),
    (col("salary") / 12).alias("monthly_salary"),
    (col("salary") * 0.2).alias("annual_bonus")
)

print("\n计算列：")
selected_2.show()

# 3. 排除特定列
cols_to_keep = [c for c in df_typed.columns if c not in ["id", "email"]]
selected_3 = df_typed.select(cols_to_keep)

print("\n排除 id 和 email：")
print(selected_3.columns)

# 4. 选择特定类型的列
numeric_cols = [field.name for field in df_typed.schema.fields 
                if isinstance(field.dataType, (IntegerType, DoubleType, FloatType))]

print(f"\n数值列: {numeric_cols}")
df_typed.select(numeric_cols).show(3)

## 第六部分：实战案例

综合运用数据清洗和转换技巧

In [ ]:
print("=" * 60)
print("实战案例：完整的数据清洗流程")
print("=" * 60)

# 步骤 1: 删除重复数据
df_step1 = df.dropDuplicates()
print(f"步骤 1 - 删除重复: {df.count()} → {df_step1.count()} 行")

# 步骤 2: 标准化字符串（小写、去空格）
df_step2 = df_step1 \
    .withColumn("email", lower(trim(col("email")))) \
    .withColumn("gender", initcap(trim(col("gender")))) \
    .withColumn("city", initcap(trim(col("city"))))

print("步骤 2 - 字符串标准化完成")

# 步骤 3: 处理异常值
df_step3 = df_step2.filter(
    (col("age").isNull() | ((col("age") >= 18) & (col("age") <= 100))) &
    (col("salary").isNull() | (col("salary") >= 0))
)

print(f"步骤 3 - 过滤异常值: {df_step2.count()} → {df_step3.count()} 行")

# 步骤 4: 填充缺失值
avg_age = df_step3.select(mean("age")).first()[0]
avg_salary = df_step3.select(mean("salary")).first()[0]

df_step4 = df_step3.fillna({
    "name": "Unknown",
    "email": "unknown@example.com",
    "age": avg_age,
    "salary": avg_salary,
    "join_date": "2023-01-01"
})

print(f"步骤 4 - 填充缺失值完成 (avg_age={avg_age:.1f}, avg_salary={avg_salary:.2f})")

# 步骤 5: 类型转换
df_step5 = df_step4 \
    .withColumn("join_date", to_date(col("join_date"))) \
    .withColumn("age", col("age").cast(IntegerType())) \
    .withColumn("salary", col("salary").cast(DoubleType()))

print("步骤 5 - 类型转换完成")

# 步骤 6: 添加派生列
df_final = df_step5 \
    .withColumn("salary_category",
                when(col("salary") >= 85000, "High")
                .when(col("salary") >= 75000, "Medium")
                .otherwise("Low")) \
    .withColumn("age_group",
                when(col("age") < 30, "20s")
                .when(col("age") < 40, "30s")
                .otherwise("40+")) \
    .withColumn("join_year", year(col("join_date"))) \
    .withColumn("email_domain", split(col("email"), "@")[1]) \
    .withColumn("monthly_salary", round(col("salary") / 12, 2))

print("步骤 6 - 添加派生列完成")

print("\n" + "=" * 60)
print("最终清洗后的数据")
print("=" * 60)

df_final.show(truncate=False)

print("\n最终数据统计：")
print(f"总行数: {df_final.count()}")
print(f"总列数: {len(df_final.columns)}")
print(f"列名: {df_final.columns}")

# 保存清洗后的数据
# df_final.write.mode("overwrite").parquet("/tmp/cleaned_employees")
# print("\n✅ 数据已保存到 /tmp/cleaned_employees")

## 第七部分：性能优化技巧

In [ ]:
print("=" * 60)
print("性能优化技巧")
print("=" * 60)

# 1. 使用 cache() 缓存频繁使用的 DataFrame
df_cached = df_final.cache()
print("✅ DataFrame 已缓存")

# 2. 查看执行计划
print("\n查询执行计划：")
df_final.filter(col("salary") > 75000).explain(mode="simple")

# 3. 使用 persist() 控制存储级别
from pyspark import StorageLevel
df_persisted = df_final.persist(StorageLevel.MEMORY_AND_DISK)
print("\n✅ DataFrame 已持久化到内存和磁盘")

# 4. 分区优化
print(f"\n当前分区数: {df_final.rdd.getNumPartitions()}")

# 重新分区（适用于大数据集）
df_repartitioned = df_final.repartition(4)
print(f"重新分区后: {df_repartitioned.rdd.getNumPartitions()} 个分区")

# 5. 性能提示
print("\n" + "=" * 60)
print("性能优化建议")
print("=" * 60)
print("""
1. 尽早过滤数据 (filter/where)
2. 选择需要的列 (select)
3. 缓存频繁使用的 DataFrame (cache/persist)
4. 合理设置分区数 (repartition/coalesce)
5. 避免使用 collect() 和 show() 在大数据集上
6. 使用 broadcast join 处理小表
7. 使用 Parquet 格式存储数据
""")

## 练习题

### 练习 1：数据清洗
创建一个包含质量问题的订单数据，并清洗它

In [ ]:
# 练习数据
order_data = [
    (1, "ORD001", "2024-01-15", 100.50, "Alice"),
    (2, "ORD002", "2024-01-16", None, "Bob"),  # 缺失金额
    (3, "ORD003", None, 200.00, "Charlie"),  # 缺失日期
    (1, "ORD001", "2024-01-15", 100.50, "Alice"),  # 重复
    (4, "ORD004", "2024-01-17", -50.00, "David"),  # 异常金额
]

orders_df = spark.createDataFrame(order_data, ["id", "order_id", "order_date", "amount", "customer"])

print("原始订单数据：")
orders_df.show()

# TODO: 完成以下任务
# 1. 删除重复订单
# 2. 过滤掉金额为负的订单
# 3. 用平均金额填充缺失的 amount
# 4. 用 '2024-01-01' 填充缺失的日期
# 5. 添加一个 amount_category 列（High: >150, Medium: 100-150, Low: <100）

# 你的代码：

### 练习 2：字符串处理
处理客户信息中的字符串数据

In [ ]:
# 练习数据
customer_data = [
    (1, "  ALICE SMITH  ", "alice.smith@GMAIL.com", "NEW YORK"),
    (2, "bob jones", "BOB@yahoo.com", "los angeles"),
    (3, "Charlie Brown", "charlie@Email.COM", "Chicago"),
]

customers_df = spark.createDataFrame(customer_data, ["id", "name", "email", "city"])

print("原始客户数据：")
customers_df.show(truncate=False)

# TODO: 完成以下任务
# 1. 去除 name 的首尾空格
# 2. 将 name 转换为标题格式（每个单词首字母大写）
# 3. 将 email 全部转换为小写
# 4. 从 email 中提取用户名和域名
# 5. 将 city 转换为标题格式

# 你的代码：

## 总结

### 本教程学到的核心技能：

#### 数据清洗
- ✅ 处理缺失值：`dropna()`, `fillna()`
- ✅ 删除重复：`dropDuplicates()`
- ✅ 异常值检测：过滤、IQR 方法

#### 数据转换
- ✅ 列操作：`withColumn()`, `drop()`, `withColumnRenamed()`
- ✅ 字符串处理：`upper()`, `lower()`, `trim()`, `split()`
- ✅ 类型转换：`cast()`, `to_date()`
- ✅ 条件逻辑：`when().otherwise()`

#### 高级技巧
- ✅ 复杂过滤：`filter()`, `isin()`, `like()`, `between()`
- ✅ 动态列选择
- ✅ 性能优化：`cache()`, `repartition()`

### 下一步学习
1. 窗口函数（排名、移动平均）
2. 复杂数据类型（数组、结构体、Map）
3. JOIN 优化技巧
4. 实战项目

In [ ]:
# 关闭 Spark Session
# spark.stop()
# print("✅ Spark Session 已关闭")

In [ ]:
# 1. 确保在正确的目录
pwd
# 应该显示：/Users/zhengzhang

# 2. （可选）清理 Notebook 输出
jupyter nbconvert --clear-output --inplace "Documents/2026/data engineer/learn pyspark/pep8.ipynb"

# 3. 查看修改了什么
git status

# 4. 查看具体改动（可选）
git diff "Documents/2026/data engineer/learn pyspark/pep8.ipynb"

# 5. 添加修改
git add "Documents/2026/data engineer/learn pyspark/pep8.ipynb"

# 6. 提交
git commit -m "Update: 更新 PEP 8 笔记内容"

# 7. 推送到 GitHub
git push

In [ ]:
PySpark_DataFrame_Advanced

SyntaxError: invalid syntax (3613438092.py, line 1)